In [ ]:
using Distributed

addprocs(4)

println(workers())

**Be careful not launching this cell multiple times as it could create an unwanted number of process on your computer**

If you want to reload the cell, please reload your kernel beforehand.

You can modify the number of process depending on your computer and its number of core.

In [ ]:
@everywhere begin
    include("../src/Conduction.jl");
    using Plots, VegaLite, LaTeXStrings, QuadGK
end


In [ ]:
semiconductor = Conduction.Semiconductor(
    1.38 * 10^-23,
    1.6*10^-19,
    10^7,
    0.1 * 1.6*10^-19,
    2.1 * 10^18,
    2.1 * 10^18,
    0.1 * 1.6 * 10^-19,
    -10^3,
    10^13,
    -10.0,
    2.7,
    2.7
);

range = -10:0.2:10
res_VRH = pmap(x -> Conduction.electronMobility(semiconductor, Conduction.RnnVRH, x, 300),
    range);
res_PERCO = pmap(x -> Conduction.electronMobility(semiconductor, Conduction.RnnPercoField,
    x, 300),
    range);

In [ ]:
plot(range, [res_VRH, res_PERCO], xlabel=L"U^\prime", ylabel=L"\mu[cm^2/(V.s)]",
    label=["VRH"  "Perco with field"])

In [ ]:
pondered_VRH = res_VRH .*
    map(x -> Conduction.DOS(semiconductor, x, 300) * Conduction.F(semiconductor, x, 300) *
        semiconductor.k * 300, range);

    pondered_PERCO = res_PERCO .*
    map(x -> Conduction.DOS(semiconductor, x, 300) * Conduction.F(semiconductor, x, 300) *
        semiconductor.k * 300, range);

plot(range, [pondered_VRH, pondered_PERCO], xlabel=L"U^\prime", ylabel=L"\mu / cm^3",
    label=["VRH" "Perco with field"])